In [1]:
#initialize code and import packages
#from run_multiscale_mapping_ts import *
import dask
from dask.distributed import LocalCluster, Client, wait
import os
import convenient as cv
from run_multiscale_mapping_ts import *
import xarray as xr
import numpy as np
import math

In [ ]:
##FOR USE ON SCRATCH
scheduler_file = os.path.join(os.environ["SCRATCH"], "scheduler_file.json")

dask.config.config["distributed"]["dashboard"]["link"] = "{JUPYTERHUB_SERVICE_PREFIX}proxy/{host}:{port}/status" 

client = Client(scheduler_file=scheduler_file)
client

In [ ]:
client.upload_file("njit_options.py")
client.upload_file("basisfunctions.py")
client.upload_file("covariancefunctions.py")
client.upload_file("distancefunctions.py")
client.upload_file("dotproduct.py")
client.upload_file("solvers.py")
client.upload_file("map_anom.py")
client.upload_file("batch_map_anom.py")
client.upload_file("distancefunctions.py")
client.upload_file("mean_field_lsq.py")
client.upload_file("negative_loglikelihood.py")
client.upload_file("maximum_likelihood.py")

In [2]:
### USE FOR TESTING ON HOMES

cluster = LocalCluster()  # Launches a scheduler and workers locally
client = Client(cluster)  # Connect to distributed cluster and override default
print('External dashboard-link: ', client.dashboard_link)
print(client)

External dashboard-link:  http://127.0.0.1:8787/status
<Client: 'tcp://127.0.0.1:42441' processes=16 threads=256, memory=476.37 GiB>


In [3]:
#window_size_betaparam = math.sqrt(1.0 * 500e3**2)
window_size_covparam = math.sqrt(2.0 * 700e3**2) # 2 x the area of a window w/ 700e3 meter radius
timewindow_size = 250 # Days

NHarm = 2
tsdatafile = '/global/homes/k/kefalc/code/data_files/Final/floats_years_2015_2019_temperature_salinity_10daily_SO_30S_edit.nc'
maskfile = '/global/homes/k/kefalc/code/gpr-mapping-covar/SO_mask.nc'
testn = 0
Nblocks = 5
nu = 1.5
Opt_nu = False
time_i = (np.datetime64('2015-01-01') - np.datetime64('1970-01-01T00:00:00', 'ns')) / np.timedelta64(1, 'ns') / 1e9 / 86400.0
time_f = (np.datetime64('2019-12-31') - np.datetime64('1970-01-01T00:00:00', 'ns')) / np.timedelta64(1, 'ns') / 1e9 / 86400.0

grid_time_i = (np.datetime64('2017-01-01') - np.datetime64('1970-01-01T00:00:00', 'ns')) / np.timedelta64(1, 'ns') / 1e9 / 86400.0
grid_time_f = (np.datetime64('2017-12-31') - np.datetime64('1970-01-01T00:00:00', 'ns')) / np.timedelta64(1, 'ns') / 1e9 / 86400.0

#outfile = '/global/homes/k/kefalc/code/'+'small_scale/'+str(np.int64(window_size_betaparam/1e3))+'_wscov'+str(np.int64(window_size_covparam/1e3))+'_plevel'+str(plevel)+'.nc'


In [ ]:
%%time
for plevel in range(5,7):
    residualfile =  '/global/homes/k/kefalc/code/anom_files/final/TS/res'+'/residualfile_plevel'+str(plevel)+'_dates'+str(time_i)+'_'+str(time_f)+ '_test'+str(testn)+'.nc'
    #'/global/homes/k/kefalc/code/anom_files/test/residualfile_plevel3_dates16436.0_18261.0_test0.nc'
    covparamfile = '/global/homes/k/kefalc/code/covar_files/final/TS/covparam'+'/covparam_ws'+str(np.int64(window_size_covparam/1e3))+'_N'+str(Nblocks)+'_plevel'+str(plevel)+'_LBFGS'+'_dates'+str(time_i)+'_'+str(time_f)+'_test'+str(testn)+'.nc'
    initcovparamfile = None
    ts_covparam( plevel, residualfile, maskfile, covparamfile, window_size_covparam, Nblocks, time_i, time_f, client, nu, initcovparamfile, Opt_nu=Opt_nu )
    TS_gridanom = '/global/homes/k/kefalc/code/covar_files/final'+'/grid_anom_ws'+str(np.int64(window_size_covparam/1e3))+'_N'+str(Nblocks)+'_plevel'+str(plevel)+'_dates'+str(grid_time_i)+'_'+str(grid_time_f)+'_test'+str(testn)+'.nc'
    #grid_anom_seasonal_ts( plevel, covparamfile, residualfile, maskfile, window_size_covparam, timewindow_size, grid_time_i, grid_time_f, TS_gridanom, client, batchsize=10 )
    

<xarray.DataArray 'pressure' ()>
array(-1000) dbar pressurelevel (pressureindex 5): 

In [ ]:
client.shutdown()

In [ ]:
##homes